In [1]:
import re
import pandas as pd
import spacy
from textblob import TextBlob
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter
import matplotlib.pyplot as plt

# Download required NLTK data
try:
    nltk.download('vader_lexicon')
except:
    pass

# Load spaCy model for NER
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Please install spaCy model: python -m spacy download en_core_web_sm")
    nlp = None

# Sample Amazon product reviews data
reviews_data = [
    {
        "review": "The Apple iPhone 14 Pro is absolutely amazing! The camera quality is outstanding and the battery life lasts all day. Highly recommend this product.",
        "product_category": "Electronics"
    },
    {
        "review": "I bought the Samsung Galaxy S23 and it's been nothing but problems. The screen froze multiple times and customer service was terrible.",
        "product_category": "Electronics"
    },
    {
        "review": "Nike Air Max shoes are incredibly comfortable for running. The quality is excellent and they look great too!",
        "product_category": "Sports & Outdoors"
    },
    {
        "review": "The Sony WH-1000XM4 headphones have terrible sound quality. Very disappointed with this purchase from Sony.",
        "product_category": "Electronics"
    },
    {
        "review": "I love my new MacBook Pro from Apple. The M2 chip is incredibly fast and the display is beautiful.",
        "product_category": "Computers"
    },
    {
        "review": "The Adidas Ultraboost running shoes are uncomfortable and caused blisters. Would not recommend.",
        "product_category": "Sports & Outdoors"
    },
    {
        "review": "Microsoft Surface Pro is a fantastic device for work. The pen functionality works perfectly.",
        "product_category": "Computers"
    },
    {
        "review": "This HP printer is constantly jamming and the ink cartridges are overpriced. Worst purchase ever.",
        "product_category": "Office Products"
    }
]

class AmazonReviewAnalyzer:
    def __init__(self):
        self.nlp = nlp
        self.sia = SentimentIntensityAnalyzer()

        # Common product and brand patterns
        self.brand_patterns = [
            r'\b(Apple|Samsung|Sony|Nike|Adidas|Microsoft|HP|Dell|Lenovo|Google|Amazon|Bose|Canon|Nikon)\b',
            r'\b(MacBook|iPhone|Galaxy|Surface|ThinkPad|Pixel|Kindle|PlayStation|Xbox)\b'
        ]

        self.product_patterns = [
            r'\b(headphones?|earbuds?|speakers?|laptop|computer|phone|smartphone|tablet|watch|shoes|printer|camera)\b',
            r'\b(running shoes|wireless headphones|gaming laptop|smart watch|bluetooth speaker)\b'
        ]

    def extract_entities_spacy(self, text):
        """Extract entities using spaCy NER"""
        if not self.nlp:
            return {"products": [], "brands": [], "orgs": []}

        doc = self.nlp(text)
        entities = {
            "products": [],
            "brands": [],
            "orgs": []
        }

        for ent in doc.ents:
            if ent.label_ in ["PRODUCT", "ORG"]:
                entity_text = ent.text.strip()
                if ent.label_ == "PRODUCT":
                    entities["products"].append(entity_text)
                else:
                    # Check if it's likely a brand
                    if any(brand.lower() in entity_text.lower() for brand in
                          ["Apple", "Samsung", "Sony", "Nike", "Microsoft", "HP"]):
                        entities["brands"].append(entity_text)
                    else:
                        entities["orgs"].append(entity_text)

        return entities

    def extract_entities_regex(self, text):
        """Extract entities using regex patterns"""
        entities = {
            "brands": [],
            "products": []
        }

        # Extract brands
        for pattern in self.brand_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            entities["brands"].extend(matches)

        # Extract products
        for pattern in self.product_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            entities["products"].extend(matches)

        # Remove duplicates and clean
        entities["brands"] = list(set(entities["brands"]))
        entities["products"] = list(set(entities["products"]))

        return entities

    def analyze_sentiment_textblob(self, text):
        """Analyze sentiment using TextBlob"""
        blob = TextBlob(text)
        polarity = blob.sentiment.polarity

        if polarity > 0.1:
            return "positive", polarity
        elif polarity < -0.1:
            return "negative", polarity
        else:
            return "neutral", polarity

    def analyze_sentiment_vader(self, text):
        """Analyze sentiment using VADER"""
        scores = self.sia.polarity_scores(text)
        compound = scores['compound']

        if compound >= 0.05:
            return "positive", compound
        elif compound <= -0.05:
            return "negative", compound
        else:
            return "neutral", compound

    def rule_based_sentiment(self, text, entities):
        """Rule-based sentiment analysis considering entities"""
        positive_words = [
            'amazing', 'outstanding', 'excellent', 'great', 'love', 'fantastic',
            'perfect', 'awesome', 'brilliant', 'wonderful', 'recommend', 'good'
        ]

        negative_words = [
            'terrible', 'problems', 'disappointed', 'uncomfortable', 'worst',
            'jamming', 'overpriced', 'froze', 'bad', 'poor', 'awful', 'hate'
        ]

        # Count positive and negative words
        pos_count = sum(1 for word in positive_words if word in text.lower())
        neg_count = sum(1 for word in negative_words if word in text.lower())

        # Simple scoring
        if pos_count > neg_count:
            return "positive", pos_count - neg_count
        elif neg_count > pos_count:
            return "negative", neg_count - pos_count
        else:
            return "neutral", 0

    def analyze_review(self, review_text):
        """Complete analysis of a single review"""
        # Entity extraction
        spacy_entities = self.extract_entities_spacy(review_text)
        regex_entities = self.extract_entities_regex(review_text)

        # Combine entities
        all_entities = {
            "brands": list(set(spacy_entities["brands"] + regex_entities["brands"])),
            "products": list(set(spacy_entities["products"] + regex_entities["products"])),
            "organizations": spacy_entities["orgs"]
        }

        # Sentiment analysis using multiple methods
        sentiment_tb, score_tb = self.analyze_sentiment_textblob(review_text)
        sentiment_vader, score_vader = self.analyze_sentiment_vader(review_text)
        sentiment_rule, score_rule = self.rule_based_sentiment(review_text, all_entities)

        # Consensus sentiment (majority vote)
        sentiments = [sentiment_tb, sentiment_vader, sentiment_rule]
        final_sentiment = max(set(sentiments), key=sentiments.count)

        return {
            "entities": all_entities,
            "sentiment": {
                "final": final_sentiment,
                "textblob": {"sentiment": sentiment_tb, "score": score_tb},
                "vader": {"sentiment": sentiment_vader, "score": score_vader},
                "rule_based": {"sentiment": sentiment_rule, "score": score_rule}
            },
            "review_length": len(review_text)
        }

def main():
    print("🔍 Amazon Product Reviews - NER and Sentiment Analysis")
    print("=" * 60)

    analyzer = AmazonReviewAnalyzer()
    results = []

    # Analyze all reviews
    for i, review_data in enumerate(reviews_data):
        print(f"\n📝 Review {i+1}:")
        print(f"   Text: {review_data['review']}")
        print(f"   Category: {review_data['product_category']}")

        analysis = analyzer.analyze_review(review_data['review'])
        results.append(analysis)

        # Display entities
        entities = analysis['entities']
        print("   🏷️  EXTRACTED ENTITIES:")
        if entities['brands']:
            print(f"      Brands: {', '.join(entities['brands'])}")
        if entities['products']:
            print(f"      Products: {', '.join(entities['products'])}")
        if entities['organizations']:
            print(f"      Organizations: {', '.join(entities['organizations'])}")

        # Display sentiment
        sentiment = analysis['sentiment']
        sentiment_icon = "😊" if sentiment['final'] == 'positive' else "😞" if sentiment['final'] == 'negative' else "😐"
        print(f"   {sentiment_icon} SENTIMENT: {sentiment['final'].upper()}")
        print(f"      TextBlob: {sentiment['textblob']['sentiment']} ({sentiment['textblob']['score']:.3f})")
        print(f"      VADER: {sentiment['vader']['sentiment']} ({sentiment['vader']['score']:.3f})")
        print(f"      Rule-based: {sentiment['rule_based']['sentiment']} (score: {sentiment['rule_based']['score']})")
        print("-" * 60)

    # Overall analysis
    print("\n📊 OVERALL ANALYSIS")
    print("=" * 60)

    # Sentiment distribution
    sentiments = [result['sentiment']['final'] for result in results]
    sentiment_counts = Counter(sentiments)

    print(f"Sentiment Distribution:")
    for sentiment, count in sentiment_counts.items():
        percentage = (count / len(results)) * 100
        print(f"   {sentiment.upper()}: {count} reviews ({percentage:.1f}%)")

    # Most mentioned brands and products
    all_brands = []
    all_products = []

    for result in results:
        all_brands.extend(result['entities']['brands'])
        all_products.extend(result['entities']['products'])

    brand_counts = Counter(all_brands)
    product_counts = Counter(all_products)

    print(f"\n🏆 Most Mentioned Brands:")
    for brand, count in brand_counts.most_common(5):
        print(f"   {brand}: {count} mentions")

    print(f"\n📦 Most Mentioned Products:")
    for product, count in product_counts.most_common(5):
        print(f"   {product}: {count} mentions")

    # Brand sentiment analysis
    print(f"\n🎯 Brand Sentiment Analysis:")
    brand_sentiments = {}

    for i, result in enumerate(results):
        review_brands = result['entities']['brands']
        sentiment = result['sentiment']['final']

        for brand in review_brands:
            if brand not in brand_sentiments:
                brand_sentiments[brand] = []
            brand_sentiments[brand].append(sentiment)

    for brand, sentiments in brand_sentiments.items():
        pos_count = sentiments.count('positive')
        neg_count = sentiments.count('negative')
        total = len(sentiments)
        pos_percentage = (pos_count / total) * 100 if total > 0 else 0

        sentiment_icon = "😊" if pos_percentage > 60 else "😐" if pos_percentage > 40 else "😞"
        print(f"   {brand}: {sentiment_icon} {pos_count}/{total} positive ({pos_percentage:.1f}%)")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


🔍 Amazon Product Reviews - NER and Sentiment Analysis

📝 Review 1:
   Text: The Apple iPhone 14 Pro is absolutely amazing! The camera quality is outstanding and the battery life lasts all day. Highly recommend this product.
   Category: Electronics
   🏷️  EXTRACTED ENTITIES:
      Brands: iPhone, Apple
      Products: camera, iPhone 14 Pro
   😊 SENTIMENT: POSITIVE
      TextBlob: positive (0.470)
      VADER: positive (0.904)
      Rule-based: positive (score: 3)
------------------------------------------------------------

📝 Review 2:
   Text: I bought the Samsung Galaxy S23 and it's been nothing but problems. The screen froze multiple times and customer service was terrible.
   Category: Electronics
   🏷️  EXTRACTED ENTITIES:
      Brands: Galaxy, Samsung
   😞 SENTIMENT: NEGATIVE
      TextBlob: negative (-0.500)
      VADER: negative (-0.310)
      Rule-based: negative (score: 3)
------------------------------------------------------------

📝 Review 3:
   Text: Nike Air Max shoes ar